### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [220]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [221]:
#Calculate the total number of schools and students
school_count = school_data_complete["school_name"].nunique()
student_count = school_data_complete["student_name"].count()

#Calculate the Total Budget
#create dataframe for school/budget
df = pd.DataFrame(school_data_complete, columns=["school_name","budget"])

#remove duplicates
df_un = df.drop_duplicates()

#sum resulting budget column
total_budget = df_un['budget'].sum()

#Calculate the average maths and reading score
avg_maths = round(school_data_complete["maths_score"].mean(),6)
avg_reading = round(school_data_complete["reading_score"].mean(),6)

#Calculate the percentage of students with a passing maths score (50 or greater)
#len code sourced and adapted from https://sparkbyexamples.com/pandas/how-to-get-pandas-count-rows-with-condition/?expand_article=1
pass_maths = len(school_data_complete[school_data_complete["maths_score"] >= 50])
pass_maths_perc = round((pass_maths/student_count)*100,6)

#Calculate the percentage of students with a passing reading score (50 or greater)
pass_reading = len(school_data_complete[school_data_complete["reading_score"] >= 50])
pass_reading_perc = round((pass_reading/student_count)*100,6)

#Calculate the percentage of students who passed maths and reading (% Overall Passing)
pass_mr = len(school_data_complete[(school_data_complete["reading_score"] >= 50) & (school_data_complete["maths_score"] >= 50)])
pass_mr_perc = round((pass_mr/student_count)*100,6)

In [222]:
#Create a dataframe to hold the above results
results = pd.DataFrame({'Total Schools': [school_count],'Total Students' : [student_count],'Total Budget' : [total_budget],
                        'Average Maths Score' : [avg_maths],'Average Reading Score' : [avg_reading],'% Passing Maths' : [pass_maths_perc],
                        '% Passing Reading' : [pass_reading_perc],'% Overall Passing' : [pass_mr_perc]})
results

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,24649428,70.338192,69.980138,86.078632,84.426857,72.808272


In [223]:
#Optional: give the displayed data cleaner formatting
#style.format code sourced and adapted from https://betterdatascience.com/style-pandas-dataframes/
results.style.format({
    "Total Budget":"${:,.2f}", 
    "Average Maths Score":"{:,.2f}", 
    "Average Reading Score":"{:,.2f}",
    "% Passing Maths":"{:,.2f}%",
    "% Passing Reading":"{:,.2f}%",
    "% Overall Passing":"{:,.2f}%"
    })

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.34,69.98,86.08%,84.43%,72.81%


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [224]:
#Group data by School name
school_groups = school_data_complete.groupby(['school_name'])

In [225]:
#set up static data columns unique to each school by using.first() for type and budget
#code sourced and adapted from https://stackoverflow.com/questions/20067636/pandas-dataframe-get-first-row-of-each-group
sch_type = school_groups["type"].first()
sch_budget = school_groups["budget"].first()

#calculate number of students
sch_stu = school_groups["student_name"].count()

#calculate scores and budget per student
sch_avg_maths = school_groups["maths_score"].mean()
sch_avg_reading = school_groups["reading_score"].mean()
stu_budget = (sch_budget/sch_stu)

#calculate percentages
sch_pass_maths = school_data_complete[school_data_complete["maths_score"] >= 50]
sch_pass_maths_gp = sch_pass_maths.groupby("school_name")
sch_pass_maths_perc = sch_pass_maths_gp["maths_score"].count() / sch_stu *100

sch_pass_read = school_data_complete[school_data_complete["reading_score"] >= 50]
sch_pass_read_gp = sch_pass_read.groupby("school_name")
sch_pass_read_perc = sch_pass_read_gp["reading_score"].count() / sch_stu *100

sch_pass_mr = school_data_complete[(school_data_complete["reading_score"] >= 50) & (school_data_complete["maths_score"] >= 50)]
sch_pass_mr_gp = sch_pass_mr.groupby(['school_name'])
sch_pass_mr_perc = (sch_pass_mr_gp['student_name'].count() / sch_stu) *100

In [226]:
#create dataframe from dictionary
sch_summary_result = pd.DataFrame({ "School Type" : sch_type, "Total Students" : sch_stu, "Total School Budget" : sch_budget,
                                   "Per Student Budget" : stu_budget, "Average Maths Score" : sch_avg_maths, "Average Reading Score" : sch_avg_reading,
                                   "% Passing Maths" : sch_pass_maths_perc, "% Passing Reading" : sch_pass_read_perc, "% Overall Passing" : sch_pass_mr_perc})

#format dataframe columns
sch_summary_result["Total School Budget"] = sch_summary_result["Total School Budget"].map("${:,.2f}".format)
sch_summary_result["Average Maths Score"] = sch_summary_result["Average Maths Score"].map("{:,.2f}".format)
sch_summary_result["Average Reading Score"] = sch_summary_result["Average Reading Score"].map("{:,.2f}".format)
sch_summary_result["% Passing Maths"] = sch_summary_result["% Passing Maths"].map("{:,.2f}%".format)
sch_summary_result["% Passing Reading"] = sch_summary_result["% Passing Reading"].map("{:,.2f}%".format)
sch_summary_result["% Overall Passing"] = sch_summary_result["% Overall Passing"].map("{:,.2f}%".format)

sch_summary_result

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",628.0,72.35,71.01,91.64%,87.38%,80.08%
Cabrera High School,Independent,1858,"$1,081,356.00",582.0,71.66,71.36,90.85%,89.07%,80.79%
Figueroa High School,Government,2949,"$1,884,411.00",639.0,68.70,69.08,81.65%,82.81%,67.65%
Ford High School,Government,2739,"$1,763,916.00",644.0,69.09,69.57,82.44%,82.22%,67.47%
Griffin High School,Independent,1468,"$917,500.00",625.0,71.79,71.25,91.21%,88.49%,81.34%
Hernandez High School,Government,4635,"$3,022,020.00",652.0,68.87,69.19,80.95%,81.88%,66.36%
Holden High School,Independent,427,"$248,087.00",581.0,72.58,71.66,89.93%,88.52%,78.92%
Huang High School,Government,2917,"$1,910,635.00",655.0,68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,4761,"$3,094,650.00",650.0,68.84,69.04,82.06%,81.98%,67.19%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [227]:
#sort top 5 performing schools in decending order
top_5_performaing = sch_summary_result.sort_values("% Overall Passing", ascending=False)
top_5_performaing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",625.0,71.79,71.25,91.21%,88.49%,81.34%
Cabrera High School,Independent,1858,"$1,081,356.00",582.0,71.66,71.36,90.85%,89.07%,80.79%
Bailey High School,Government,4976,"$3,124,928.00",628.0,72.35,71.01,91.64%,87.38%,80.08%
Wright High School,Independent,1800,"$1,049,400.00",583.0,72.05,70.97,91.78%,86.67%,79.72%
Rodriguez High School,Government,3999,"$2,547,363.00",637.0,72.05,70.94,90.80%,87.40%,79.42%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [228]:
#sort lowest 5 performing schools in acending order
low_5_performaing = sch_summary_result.sort_values("% Overall Passing", ascending=True)
low_5_performaing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",652.0,68.87,69.19,80.95%,81.88%,66.36%
Huang High School,Government,2917,"$1,910,635.00",655.0,68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,4761,"$3,094,650.00",650.0,68.84,69.04,82.06%,81.98%,67.19%
Wilson High School,Independent,2283,"$1,319,574.00",578.0,69.17,68.88,82.79%,81.30%,67.46%
Ford High School,Government,2739,"$1,763,916.00",644.0,69.09,69.57,82.44%,82.22%,67.47%


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [229]:
#create year level series
year_9 = school_data_complete[school_data_complete["year"] == 9]
year_10 = school_data_complete[school_data_complete["year"] == 10]
year_11 = school_data_complete[school_data_complete["year"] == 11]
year_12 = school_data_complete[school_data_complete["year"] == 12]

#group year levels by school
year_9_gp = year_9.groupby(["school_name"])
year_10_gp = year_10.groupby(["school_name"])
year_11_gp = year_11.groupby(["school_name"])
year_12_gp = year_12.groupby(["school_name"])

#average maths scores by year level
year_9_gp_avg = year_9_gp['maths_score'].mean()
year_10_gp_avg = year_10_gp['maths_score'].mean()
year_11_gp_avg = year_11_gp['maths_score'].mean()
year_12_gp_avg = year_12_gp['maths_score'].mean()


In [230]:
#create dataframe of average maths scores
maths_scores_by_school = pd.DataFrame({ "Year 9" : year_9_gp_avg, 
                                       "Year 10" : year_10_gp_avg,
                                       "Year 11" : year_11_gp_avg,
                                       "Year 12" : year_12_gp_avg})

#format year level columns
maths_scores_by_school["Year 9"] = maths_scores_by_school["Year 9"].map("{:,.2f}".format)
maths_scores_by_school["Year 10"] = maths_scores_by_school["Year 10"].map("{:,.2f}".format)
maths_scores_by_school["Year 11"] = maths_scores_by_school["Year 11"].map("{:,.2f}".format)
maths_scores_by_school["Year 12"] = maths_scores_by_school["Year 12"].map("{:,.2f}".format)

maths_scores_by_school

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.49,71.90,72.37,72.68
Cabrera High School,72.32,72.44,71.01,70.60
Figueroa High School,68.48,68.33,68.81,69.33
Ford High School,69.02,69.39,69.25,68.62
Griffin High School,72.79,71.09,71.69,71.47
Hernandez High School,68.59,68.87,69.15,68.99
Holden High School,70.54,75.11,71.64,73.41
Huang High School,69.08,68.53,69.43,68.64
Johnson High School,69.47,67.99,68.64,69.29


## Reading Score by Year

* Perform the same operations as above for reading scores

In [231]:
#average reading scores by year level
year_9_rgp_avg = year_9_gp['reading_score'].mean()
year_10_rgp_avg = year_10_gp['reading_score'].mean()
year_11_rgp_avg = year_11_gp['reading_score'].mean()
year_12_rgp_avg = year_12_gp['reading_score'].mean()

In [232]:
#create dataframe of average reading scores
reading_scores_by_school = pd.DataFrame({ "Year 9" : year_9_rgp_avg, 
                                       "Year 10" : year_10_rgp_avg,
                                       "Year 11" : year_11_rgp_avg,
                                       "Year 12" : year_12_rgp_avg})

#format year level columns
reading_scores_by_school["Year 9"] = reading_scores_by_school["Year 9"].map("{:,.2f}".format)
reading_scores_by_school["Year 10"] = reading_scores_by_school["Year 10"].map("{:,.2f}".format)
reading_scores_by_school["Year 11"] = reading_scores_by_school["Year 11"].map("{:,.2f}".format)
reading_scores_by_school["Year 12"] = reading_scores_by_school["Year 12"].map("{:,.2f}".format)

reading_scores_by_school

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.90,70.85,70.32,72.20
Cabrera High School,71.17,71.33,71.20,71.86
Figueroa High School,70.26,67.68,69.15,69.08
Ford High School,69.62,68.99,70.74,68.85
Griffin High School,72.03,70.75,72.39,69.43
Hernandez High School,68.48,70.62,68.42,69.24
Holden High School,71.60,71.10,73.31,70.48
Huang High School,68.67,69.52,68.74,68.67
Johnson High School,68.72,69.30,69.97,67.99


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [233]:
#create bin for spending ranges
bins = [0, 584, 629, 644, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

#recreate dataframe without formatting
sch_results = pd.DataFrame({"Per Student Budget" : stu_budget,"Average Maths Score" : sch_avg_maths, "Average Reading Score" : sch_avg_reading, "% Passing Maths" : sch_pass_maths_perc, 
                            "% Passing Reading" : sch_pass_read_perc, "% Overall Passing" : sch_pass_mr_perc})

#slice data and place into bins
sch_results["Spending Ranges (Per Student)"] = pd.cut(sch_results["Per Student Budget"], bins, labels=group_names, include_lowest=True)

In [234]:
#create group based on bins
spending_result = sch_results.groupby("Spending Ranges (Per Student)")

#calculate scores and percentages based on spending
avg_maths_spend = spending_result["Average Maths Score"].mean()
avg_read_spend = spending_result["Average Reading Score"].mean()
pass_maths_spend = spending_result["% Passing Maths"].mean()
pass_read_spend = spending_result["% Passing Reading"].mean()
pass_overall_spend = spending_result["% Overall Passing"].mean()

sch_bins_result = pd.DataFrame({"Average Maths Score" : avg_maths_spend,
                                "Average Reading Score" : avg_read_spend,
                                "% Passing Maths" : pass_maths_spend,
                                "% Passing Reading" : pass_read_spend,
                                "% Overall Passing" : pass_overall_spend})

#format results
sch_bins_result["Average Maths Score"] = sch_bins_result["Average Maths Score"].map("{:,.2f}".format)
sch_bins_result["Average Reading Score"] = sch_bins_result["Average Reading Score"].map("{:,.2f}".format)
sch_bins_result["% Passing Maths"] = sch_bins_result["% Passing Maths"].map("{:,.2f}".format)
sch_bins_result["% Passing Reading"] = sch_bins_result["% Passing Reading"].map("{:,.2f}".format)
sch_bins_result["% Overall Passing"] = sch_bins_result["% Overall Passing"].map("{:,.2f}".format)

sch_bins_result

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


## Scores by School Size

* Perform the same operations as above, based on school size.

In [235]:
#create bin for spending ranges
size_bins = [0, 999, 1999, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#recreate dataframe without formatting
sch_results = pd.DataFrame({"Total Students" : sch_stu, "Average Maths Score" : sch_avg_maths, "Average Reading Score" : sch_avg_reading,
                            "% Passing Maths" : sch_pass_maths_perc, "% Passing Reading" : sch_pass_read_perc, "% Overall Passing" : sch_pass_mr_perc})

#slice data and place into bins
sch_results["School Size"] = pd.cut(sch_results["Total Students"], size_bins, labels=size_group_names, include_lowest=True)

In [239]:
#create group based on bins
size_result = sch_results.groupby("School Size")

#calculate scores and percentages based on spending
avg_maths_size = size_result["Average Maths Score"].mean()
avg_read_size = size_result["Average Reading Score"].mean()
pass_maths_size = size_result["% Passing Maths"].mean()
pass_read_size = size_result["% Passing Reading"].mean()
pass_overall_size = size_result["% Overall Passing"].mean()

#create dataframe of results
size_bins_result = pd.DataFrame({"Average Maths Score" : avg_maths_size,
                                "Average Reading Score" : avg_read_size,
                                "% Passing Maths" : pass_maths_size,
                                "% Passing Reading" : pass_read_size,
                                "% Overall Passing" : pass_overall_size})

#format results
size_bins_result["Average Maths Score"] = size_bins_result["Average Maths Score"].map("{:,.2f}".format)
size_bins_result["Average Reading Score"] = size_bins_result["Average Reading Score"].map("{:,.2f}".format)
size_bins_result["% Passing Maths"] = size_bins_result["% Passing Maths"].map("{:,.2f}".format)
size_bins_result["% Passing Reading"] = size_bins_result["% Passing Reading"].map("{:,.2f}".format)
size_bins_result["% Overall Passing"] = size_bins_result["% Overall Passing"].map("{:,.2f}".format)

size_bins_result

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.64,90.81,87.56,79.07
Medium (1000-2000),71.42,70.72,89.85,86.71,78.04
Large (2000-5000),69.75,69.58,84.25,83.30,70.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [241]:
#recreate dataframe without formatting
sch_summary_result = pd.DataFrame({ "School Type" : sch_type, "Total Students" : sch_stu, "Total School Budget" : sch_budget,
                                   "Per Student Budget" : stu_budget, "Average Maths Score" : sch_avg_maths, "Average Reading Score" : sch_avg_reading,
                                   "% Passing Maths" : sch_pass_maths_perc, "% Passing Reading" : sch_pass_read_perc, "% Overall Passing" : sch_pass_mr_perc})

#group by schol type
sch_type_gp = sch_summary_result.groupby(["School Type"])

#calculate scores and percentages based on school type
avg_maths_type = sch_type_gp["Average Maths Score"].mean()
avg_read_type = sch_type_gp["Average Reading Score"].mean()
pass_maths_type = sch_type_gp["% Passing Maths"].mean()
pass_read_type = sch_type_gp["% Passing Reading"].mean()
pass_overall_type = sch_type_gp["% Overall Passing"].mean()

#create dataframe of results
type_result = pd.DataFrame({"Average Maths Score" : avg_maths_type,
                                "Average Reading Score" : avg_read_type,
                                "% Passing Maths" : pass_maths_type,
                                "% Passing Reading" : pass_read_type,
                                "% Overall Passing" : pass_overall_type})

type_result

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
